In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
from tqdm import tqdm
sys.path.append('../Deep Learning/')
from nn_optimization_methods import SGD
from layers.Dense import Dense
from layers.Input import Input
sys.path.append('../tools/')
import tools

%matplotlib inline

## DNN from scratch

approximate sin function from random values.

In [2]:
x = np.linspace(-10, 10, 1000)

In [3]:
y = np.sin(x)**2

In [4]:
df = pd.DataFrame({'x':x,'y':y})

In [18]:
class DNN():
    
    """
    Deep Neural Network Template for regression and classification tasks
    Parameters:
        X: numpy array() data matrix
        y: numpy array() response variables, must be numeric
        output: numpy array() outputs
        outlayer = output layer
        head = input layer
    
    Use like sequential model from Keras:
        Ex: add a dense layer with 200 neurons and a sigmoid activation function:
                dnn.add("dense","sigmoid",20)
    """
    def __init__(self,X,y):
            
        self.X = X
        self.y = y
        self.output = 0
        self.outlayer = Input(self.X)
        self.head = self.outlayer
    
    def add(self,layertype,activation,neurons):
        
        if layertype == "dense":
                layer = Dense(self.outlayer,neurons,activation)
                self.outlayer.setNext(layer)
                self.outlayer = layer
            
    def foward(self,inputs=None):
        
        if np.all(inputs != None):
            
            self.head.update(inputs)
            
        layer = self.head
        
        while np.all(layer.next != None):
            
            layer.foward()
            layer = layer.getNext()
        
        layer.foward()
        return layer.output
        
    def backward_pass(self,predictions,y,loss="MSE"):
        
        cache = 0
        gradients = []
        
        if loss == "MSE":
            
            layer = self.outlayer
            cache = np.array(np.sum(predictions - y))
            derivative = layer.getPrev().dot(cache*(layer.deriv),True)
            gradients.append(derivative)
            layer = layer.getPrev()
            
            while np.all(layer.getPrev() != None):
                cache = (cache.dot(layer.getNext().getWeights().T))*layer.deriv
                gradients.append(layer.getPrev().dot(cache,True))
                layer = layer.getPrev()
            
            return gradients[::-1]
        
    def train(self,X,y,optimizer=SGD,lr=0.0001,epochs=100,batch_size=1):
        
        optimizer(self,X,y,learning_rate=lr,epochs=epochs,batch_size=batch_size)
        
    def predict(self,X):
        
        return self.foward(X)
        
    def __str__(self):
        
        string = "----------------------------- Model -----------------------------" + '\n'
        layer = self.head
        
        while np.all(layer != None):
            string += layer.__str__()
            layer = layer.getNext()
            
        return string
    

In [19]:
x = x.reshape(x.shape[0],1)
dnn = DNN(x,y)
dnn.add("dense","sigmoid",24)
dnn.add("dense","sigmoid",48)
dnn.add("dense","sigmoid",16)
dnn.add("dense","linear",1)
print(dnn)

----------------------------- Model -----------------------------
Input: shape: (None, 1)
-----------------------------------------------------------------
Dense: activation: sigmoid, weight shape: (1, 24), output shape: (None, 24), parameters: 24
-----------------------------------------------------------------
Dense: activation: sigmoid, weight shape: (24, 48), output shape: (None, 48), parameters: 1152
-----------------------------------------------------------------
Dense: activation: sigmoid, weight shape: (48, 16), output shape: (None, 16), parameters: 768
-----------------------------------------------------------------
Dense: activation: linear, weight shape: (16, 1), output shape: (None, 1), parameters: 16
-----------------------------------------------------------------



In [ ]:
dnn.train(x,y,lr=0.0001,epochs=2500)




  0%|          | 0/2500 [00:00<?, ?it/s]


  0%|          | 1/2500 [00:00<33:17,  1.25it/s]

MSE: 1.4606785737472456





  0%|          | 2/2500 [00:01<35:41,  1.17it/s]

MSE: 0.6478635514836445





  0%|          | 3/2500 [00:02<36:19,  1.15it/s]

MSE: 0.3461528958652822


In [ ]:
predictions = dnn.predict(x)

In [ ]:
predictions = predictions.reshape(predictions.shape[0],)

In [ ]:
newd = pd.DataFrame({'x':df["x"],'y':predictions})

Predicted after 10000 epochs

In [ ]:
sns.lmplot(x='x', y='y',data=newd,
           fit_reg=False, # No regression line
          )  

Original sin wave

In [ ]:
sns.lmplot(x='x', y='y',data=df,
           fit_reg=False, # No regression line
          )   